# Step One: Use Existing Labels

In [1]:
import glob as glob
import os
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
from sklearn.metrics import (
    classification_report,
    cohen_kappa_score,
    confusion_matrix,
    roc_auc_score,
)

import image_processor

In [2]:
useful_labels = [
    "bottle",
    "fork",
    "knife",
    "spoon",
    "bowl",
    "banana",
    "apple",
    "sandwich",
    "orange",
    "broccoli",
    "carrot",
    "hot dog",
    "pizza",
    "donut",
    "cake",
    "diningtable",
    "tvmonitor",
    "laptop",
    "mouse",
    "remote",
    "keyboard",
    "cell phone",
    "microwave",
    "oven",
    "toaster",
    "sink",
    "refrigerator",
    "book",
]

processor = image_processor.ImageProcessor(
    labels_path="/Users/tasanders/Google Drive/Square Eyes (DP20)/5 - Data collection and management/Image Blurring Test/Yolo/coco.names",
    config_path="/Users/tasanders/Google Drive/Square Eyes (DP20)/5 - Data collection and management/Image Blurring Test/Yolo/yolov3.cfg",
    weights_path="/Users/tasanders/Google Drive/Square Eyes (DP20)/5 - Data collection and management/Image Blurring Test/Yolo/yolov3.weights",
    useful_labels=useful_labels,
    image_folder="/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT/YOLO_Training/Images",
    training_out="/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT/YOLO_Training/Output_New",
    annotimage_out="/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT/YOLO_Training/Output_New",
)

In [3]:
%%time
processor.run_folder(parallel=True)

CPU times: user 154 ms, sys: 185 ms, total: 339 ms
Wall time: 1h 21min 52s


##  Test against Bridget's coding

In [ ]:
%%time
coded_data = pd.read_csv(
    "/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT/Coding Framework Test Images/Screen Time Coding Data - Device.csv"
)
folder = (
    "/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT/Coding Framework Test Images"
)
df = predict_and_save(folder, coded_data)

In [ ]:
df_backup = df.copy()
df.to_csv(
    "/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT/Coding Framework Test Images/YOLO.csv",
    index=False,
)

In [ ]:
cat_maybe = [
    "mouse",
    "remote",
    "keyboard",
]

cat_def = [
    "tvmonitor",
    "laptop",
    "cell phone",
]

In [ ]:
df["screen_def"] = np.where(df["prediction"].isin(cat_def), 1, 0)
df["screen_maybe"] = np.where(df["prediction"].isin(cat_def + cat_maybe), 1, 0)

In [ ]:
def conf_def(df, confthresh):
    if df["confidence"] > confthresh and df["screen_def"] == 1:
        return 1
    else:
        return 0


def conf_maybe(df, confthresh):
    if df["confidence"] > confthresh and df["screen_maybe"] == 1:
        return 1
    else:
        return 0

In [ ]:
for x in np.arange(0.2, 0.51, 0.05):
    df["screen_def_" + str(x)] = df.apply(conf_def, confthresh=x, axis=1)
    df["screen_maybe_" + str(x)] = df.apply(conf_maybe, confthresh=x, axis=1)

In [ ]:
df = df.merge(coded_data, left_on="image", right_on="filename")
df.drop(columns="filename", inplace=True)
df[["device", "device_excl_bkg"]] = df[["device", "device_excl_bkg"]].astype(int)
df = df.drop(columns=["prediction", "confidence", "id"]).groupby(["image"]).any()

In [ ]:
df.columns

In [ ]:
true_devices = ["device", "device_excl_bkg"]
predicted_devices = df.drop(
    columns=["device", "device_excl_bkg", "screen_def", "screen_maybe",]
)

In [ ]:
for true_device in true_devices:
    for predicted_device in predicted_devices:
        print(f"Comparing: {true_device} & {predicted_device}")
        print(classification_report(df[true_device], df[predicted_device]))

In [ ]:
print(f"Comparing: device & screen_maybe_0.3")
print(classification_report(df["device"], df["screen_maybe_0.3"]))
print(f"AUC: {roc_auc_score(df['device'], df['screen_maybe_0.3'])}")
print(f"Comparing: device_excl_bkg & screen_maybe_0.3")
print(classification_report(df["device_excl_bkg"], df["screen_maybe_0.3"]))
print(cohen_kappa_score(df["device_excl_bkg"], df["screen_maybe_0.3"]))
print(f"AUC: {roc_auc_score(df['device_excl_bkg'], df['screen_maybe_0.3'])}")
print(f"Comparing: device & screen_maybe_0.2")
print(classification_report(df["device"], df["screen_maybe_0.2"]))
print(f"AUC: {roc_auc_score(df['device'], df['screen_maybe_0.2'])}")
print(f"Comparing: device_excl_bkg & screen_maybe_0.2")
print(classification_report(df["device"], df["screen_maybe_0.2"]))
print(f"AUC: {roc_auc_score(df['device'], df['screen_maybe_0.2'])}")